In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import matplotlib.pyplot as plt

from pathlib import Path
import os
import sys

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import torch
from torch import nn
from torch import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchsummary import summary
import safetensors

from transformers import WhisperTokenizer, WhisperFeatureExtractor
from transformers import GenerationConfig
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# import LORA modules
from peft import LoraConfig
from peft import LoraModel, AdaLoraModel, PeftModel, get_peft_model

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, QuantileTransformer

from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
metric = evaluate.load("wer")

from pathlib import Path
with open("../default_paths.txt", 'r') as file:
    lines = file.readlines()
    data_path    = Path(lines[0].strip().split("=")[1])
    project_path = Path(lines[1].strip().split("=")[1])
    
utils_path = project_path / "utils"
train_path = data_path / "train"
test_path  = data_path / "test"
sys.path.insert(0, str(utils_path))

from data_utils_lesha import (process_all_files, 
process_string, 
save_sentences_to_txt,
load_sentences_from_txt,
RegularizeConfig,
ModelAdaptationConfig,
PreprocessConfig,
AugmentConfig,
configure_input_layers,
configure_learnable_layers,
WhisperAugmentDataset,
DataCollatorSpeechSeq2SeqWithPadding,
count_parameters, 
save_config, 
load_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
WHISPER_MODEL_NAME = "openai/whisper-large-v3"

prev_experiment_path = data_path / "experiments" / WHISPER_MODEL_NAME / "GRADUAL_FULL_dropout"
prev_checkpoint_path = prev_experiment_path / "checkpoint-6600"

regularize_config = RegularizeConfig().from_json_file(prev_experiment_path / "regularize_config.json")
adaptation_config = ModelAdaptationConfig().from_json_file(prev_experiment_path / "adaptation_config.json")
preprocess_config = PreprocessConfig().from_json_file(prev_experiment_path / "preprocess_config.json")
augment_config    = AugmentConfig().from_json_file(prev_experiment_path / "augment_config.json")

In [3]:
adaptation_config.lora_config

{'peft_type': 'LORA',
 'auto_mapping': None,
 'base_model_name_or_path': None,
 'revision': None,
 'task_type': 'SEQ_2_SEQ_LM',
 'inference_mode': False,
 'r': 8,
 'target_modules': None,
 'lora_alpha': 8,
 'lora_dropout': 0.0,
 'fan_in_fan_out': False,
 'bias': 'none',
 'use_rslora': True,
 'modules_to_save': None,
 'init_lora_weights': True,
 'layers_to_transform': None,
 'layers_pattern': None,
 'rank_pattern': {},
 'alpha_pattern': {},
 'megatron_config': None,
 'megatron_core': 'megatron.core',
 'loftq_config': {},
 'use_dora': True,
 'layer_replication': None}

In [ ]:
# load feature/label processing engines|
feature_extractor = WhisperFeatureExtractor.from_pretrained(WHISPER_MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(WHISPER_MODEL_NAME, task="transcribe", language="english")
model = WhisperForConditionalGeneration.from_pretrained(WHISPER_MODEL_NAME, **regularize_config.to_dict())

if ".en" not in WHISPER_MODEL_NAME:
    model.generation_config.language = "english"
    model.generation_config.task = "transcribe"
    model.generation_config.forced_decoder_ids = None


adaptation_config.adapt_model="full"

# adapt model according to adaptation config
model = configure_input_layers(model, adaptation_config)
model = configure_learnable_layers(model, adaptation_config)
model